In [1]:
%%time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os
from REINFORCE_soft import MultiStockEnv, Agent, play_one_episode, maybe_make_dir
import market_tickers
from model_config import model_parameters
import tensorflow as tf

CPU times: user 2.85 s, sys: 1.95 s, total: 4.79 s
Wall time: 4.43 s


In [2]:
#------ small test-sweep ------#
GAMMA_TRADES = [0.1, 1, 10, 60, 100]
GAMMA_RISKS = [0.1, 1, 18, 56, 100, 562, 1000]
#------ extended-boyd-sweep ------#
# GAMMA_TRADES =  [0.1, 0.5, 1, 2, 3, 4, 5, 5.5, 6, 6.5, 7, 7.5, 8, 9, 10, 11, 12, 15, 20, 30, 45, 60, 100, 200]
# GAMMA_RISKS = [0.1, 0.178, 0.316, 0.562, 1, 2, 3, 6, 10, 18, 32, 56, 100, 178, 316, 562, 1000, 2000, 5000, 10000, 20000]
#---------------------------------#

GAMMA_PAIRS = []
for tr in GAMMA_TRADES:
    for ri in GAMMA_RISKS:
        GAMMA_PAIRS.append((tr, ri))
        
#format: (gamma_trade, gamma_risk)
print('number of pairs: ', len(GAMMA_PAIRS))
#GAMMA_PAIRS

number of pairs:  35


In [3]:
# papermill parameters
SEED_LIST = [
             0,
             #1111,
             #2222, 
             #3333, 
             #4444, 
             #5555,
             #6666,
             #7777,
             #8888,
             #9999
            ]

#TICKERS = market_tickers.DOW_30_TICKER
TICKERS = ['AAPL', 'AMD', 'CSCO', 'F', 'INTC']
MARKET_NAME = 'TEST_5'
MODEL_BASE_NAME = 'RL_CNN' # use either | 'RL_CNN' | 'RL_str_fcast' | 'RL_all_inp' |
FROM = '2017-01-03' # start of training set
UNTIL = '2019-01-01' # end of training set
NB_EPISODES = 200 # number of episodes to train for
SAVE_EVERY = 200 # (used 100 when not from tretrained) save model weights every time this amount of episodes pass

In [4]:
# other constants and parameters
TICKERS.sort()
FILE_PERIOD = '1d' # weekly='5d', daily='1d'
DAYS_IN_EPISODE = 30 # 365 for one-year long episodes (conditions checked at end of episode)
EPISODE_DRAW_DISTRIBUTION = 'uniform' # 'uniform' or 'geometric'. select starting point of eposide according to this distribution when generated
HALF_SPREAD = 0.0005/2.0 # 'a' in transaction cost function
NONLIN_COEFF = 1.0 # 'b' transaction cost function
POWER = 1.5 # power for change in poertfolio vector used in transaction cost
#GAMMA_RISK, GAMMA_TRADE, GAMMA_HOLD = 18, 6.5, 0.0 # relative importance of risk, trading cost, and holding cost
INIT_PORTFOLIO = 100000000.0 # initial portfolio value
#model_name = f'REINFORCE_CNN' #_seeded_{UNTIL} # give model a name to distinguish saved files
#NB_EPISODES = 300 #2000
MODE = 'train' # train or test mode
#FROM_PRETRAINED = False
DATA_DIR = f'../../data/{MARKET_NAME}/preprocessed_data/'

# Tune and double-check these before every test:
USE_FORECASTS = model_parameters[MODEL_BASE_NAME]['use_forecasts'] # whether to use forecasts as inputs to agent policy net or not
NB_FORECASTS = model_parameters[MODEL_BASE_NAME]['nb_forecasts'] # number of forecasts to use in state. Set to None for no forecasts
FORECAST_TYPE = model_parameters[MODEL_BASE_NAME]['forecast_type'] # use 'strong' or 'weak' forecasts (if no forecasts given, this does not matter)
USE_CNN_STATE = model_parameters[MODEL_BASE_NAME]['use_cnn_state'] # whether to pass log-rets block for CNN part of policy net (set to false for forecast only)

In [5]:
%%time
##########################
### main training loop ###
##########################

# define investor preferences to train for
#------ extended-boyd-sweep ------#
#GAMMA_TRADES = [5.5, 6, 6.5, 7, 7.5, 8, 9, 10, 11, 12, 15, 20, 30]
#GAMMA_TRADES = [4, 5]
#GAMMA_TRADES = [0.1, 0.5, 1, 2, 3, 45, 60, 100, 200]
#GAMMA_RISKS = [0.1, 0.178, 0.316, 0.562, 1, 2, 3, 6, 10, 18, 32, 56, 100, 178, 316, 562, 1000, 2000, 5000, 10000, 20000]
#---------------------------------#
counter = 0

for RANDOM_SEED in SEED_LIST:
    model_name = f'{MODEL_BASE_NAME}_seed_{RANDOM_SEED}'
    maybe_make_dir(f'experiments/{MARKET_NAME}/seeded/{MODEL_BASE_NAME}/seed_{RANDOM_SEED}')
    
    for (GAMMA_TRADE, GAMMA_RISK) in GAMMA_PAIRS:
        # set random generator seed for tf and np
        # this is probably unnecessary because it is done when env is initialised but just incase
        tf.random.set_seed(RANDOM_SEED)
        np.random.seed(RANDOM_SEED)
        counter += 1

        # check if test lready done. skip if so
        #if not FROM_PRETRAINED:
            #skip = os.path.exists(f'experiments/12_assets/00_seeded/seed_{RANDOM_SEED}/trade_{GAMMA_TRADE}_risk_{GAMMA_RISK}') # check if already exists
        #else:
        
        skip = os.path.exists(f'experiments/{MARKET_NAME}/seeded/{MODEL_BASE_NAME}/seed_{RANDOM_SEED}/trade_{GAMMA_TRADE}_risk_{GAMMA_RISK}')

        if not skip: # if not exists, do test

            print('#############################################################################')
            print(f'##\t[{counter}/{len(GAMMA_PAIRS)}]\tmodel={MODEL_BASE_NAME},\tSEED={RANDOM_SEED},\ttrade={GAMMA_TRADE},\trisk={GAMMA_RISK}\t##')
            print('#############################################################################')        

            # create agent and environment
            env = MultiStockEnv(tickers=TICKERS, 
                                from_date=FROM,
                                until=UNTIL, 
                                #nb_episodes=100,
                                cash_key='USDOLLAR', 
                                gamma_risk=GAMMA_RISK, 
                                gamma_trade=GAMMA_TRADE,
                                half_spread=HALF_SPREAD, 
                                nonlin_coef=NONLIN_COEFF, 
                                power=POWER, 
                                datadir=DATA_DIR, 
                                state_lookback_window=20, 
                                distribution=EPISODE_DRAW_DISTRIBUTION,
                                days_duration=DAYS_IN_EPISODE, 
                                mode='train', 
                                random_seed=RANDOM_SEED,
                                init_portfolio=INIT_PORTFOLIO, 
                                period_in_file_name=FILE_PERIOD, 
                                nb_forecasts=NB_FORECASTS, 
                                forecast_type=FORECAST_TYPE,
                                use_CNN_state=USE_CNN_STATE,
                                verbose=False)

            agent = Agent(alpha=0.001, 
                          gamma=0.99, 
                          n_assets=len(TICKERS)+1, 
                          tau=5, 
                          lookback_window=20, 
                          n_feature_maps=len(TICKERS)+1,
                          use_forecasts=USE_FORECASTS,
                          use_CNN_state=USE_CNN_STATE)

            # initialise policy by one forward pass then load pretrained weights
            #if FROM_PRETRAINED: 
                #agent.choose_action(env._get_obs())
                #agent.load(pretrined_weights_dir)

            # configure folders
            experiments_folder = f'experiments/{MARKET_NAME}'
            from_seeded_folder = f'{experiments_folder}/seeded'
            base_model_folder = f'{from_seeded_folder}/{MODEL_BASE_NAME}'
            seed_folder = f'{base_model_folder}/seed_{RANDOM_SEED}'
            #if FROM_PRETRAINED:
            #    pretrained_folder = f'{nb_assets_folder}/0_further_trained'
            #    preference_folder = f'{pretrained_folder}/trade_{GAMMA_TRADE}_risk_{GAMMA_RISK}'
            #else:
            preference_folder = f'{seed_folder}/trade_{GAMMA_TRADE}_risk_{GAMMA_RISK}'
            models_folder = f'{preference_folder}/models' # where saved models will be saved
            losses_folder = f'{preference_folder}/losses' # where losses will be saved for plotting training progress
            fig_folder = f'{preference_folder}/figures'

            # create dirictories for saving outputs to (if they don't exist already)
            #maybe_make_dir(nb_assets_folder)
            maybe_make_dir(seed_folder)
            maybe_make_dir(preference_folder)
            maybe_make_dir(models_folder)
            maybe_make_dir(losses_folder)
            maybe_make_dir(fig_folder)

            if MODE == 'train':
                # play the game num_episodes times and update weights according to loss
                losses = []
                for e in range(NB_EPISODES):

                    # save model weights every few episodes
                    if (e%SAVE_EVERY == 0) and (e!=0):
                        agent.save(f'{models_folder}/{model_name}_{e}')
                        # save losses of training period
                        np.save(f'{losses_folder}/{model_name}_{MODE}_{e}_losses.npy', np.array(losses))

                    t0 = datetime.datetime.now()
                    try:
                        loss = play_one_episode(agent, env, MODE)
                    except Exception as err:
                        print(f'\t*** error in episode {e}:', err)
                        loss = np.nan#5.0 # just an arbitraty number that seems high compared to usual losses
                    dt = datetime.datetime.now() - t0
                    losses.append(loss) # append episode loss
                    # print progress
                    if (e%10 == 0):
                        print(f"episode: {e + 1}/{NB_EPISODES}, loss: {loss:.4f}, duration: {dt}")


                # save the weights and losses when done	
                agent.save(f'{models_folder}/{model_name}_{NB_EPISODES}')
                np.save(f'{losses_folder}/{model_name}_{MODE}_{NB_EPISODES}_losses.npy', np.array(losses))

            # plot losses
            save_fig = True

            # load losses file
            file = f'{losses_folder}/{model_name}_{MODE}_{NB_EPISODES}_losses.npy'
            loaded_losses = np.load(file)

            # plot losses
            plt.figure(figsize=(14,5))
            plt.plot(loaded_losses)
            plt.title(f'{model_name} Training Loss')
            plt.xlabel('Episode Number')
            plt.ylabel(r'Loss: $-G \approx -\mathbb{E}\left[ \sum_{k=0}^{T} \gamma^k R_t \right]$')
            #plt.yscale('symlog')
            if save_fig:
                plt.savefig(f'{fig_folder}/{model_name}_losses.png', dpi=150, facecolor=None, edgecolor=None, bbox_inches='tight')
            plt.clf()

        else:
            print(f'\talready exists: trade_{GAMMA_TRADE}_risk_{GAMMA_RISK}')
            print('\tskip to next.')

#############################################################################
##	[1/35]	model=RL_CNN,	SEED=0,	trade=0.1,	risk=0.1	##
#############################################################################
episode: 1/200, loss: 0.0050, duration: 0:00:00.505505
episode: 11/200, loss: -0.0266, duration: 0:00:00.360063
episode: 21/200, loss: 0.0276, duration: 0:00:00.357516
episode: 31/200, loss: -0.0561, duration: 0:00:00.399690
episode: 41/200, loss: 0.0090, duration: 0:00:00.367047
episode: 51/200, loss: -0.0514, duration: 0:00:00.355037
episode: 61/200, loss: -0.0109, duration: 0:00:00.365439
episode: 71/200, loss: 0.0075, duration: 0:00:00.357139
episode: 81/200, loss: 0.0189, duration: 0:00:00.359105
episode: 91/200, loss: -0.0218, duration: 0:00:00.376011
episode: 101/200, loss: -0.0296, duration: 0:00:00.370186
episode: 111/200, loss: -0.0577, duration: 0:00:00.366908
episode: 121/200, loss: -0.0729, duration: 0:00:00.366931
episode: 131/200, loss: -0.0017, duration: 0:00:00.

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:134: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


#############################################################################
##	[22/35]	model=RL_CNN,	SEED=0,	trade=60,	risk=0.1	##
#############################################################################
episode: 1/200, loss: 0.0661, duration: 0:00:00.418241
episode: 11/200, loss: -0.0057, duration: 0:00:00.370861
episode: 21/200, loss: 0.1172, duration: 0:00:00.373151
episode: 31/200, loss: -0.0080, duration: 0:00:00.399646
episode: 41/200, loss: 0.0242, duration: 0:00:00.369865
episode: 51/200, loss: -0.0407, duration: 0:00:00.376287
episode: 61/200, loss: 0.0000, duration: 0:00:00.375389
episode: 71/200, loss: 0.0155, duration: 0:00:00.392451
episode: 81/200, loss: 0.0819, duration: 0:00:00.356048
episode: 91/200, loss: -0.0098, duration: 0:00:00.398250
episode: 101/200, loss: -0.0212, duration: 0:00:00.368225
episode: 111/200, loss: -0.0480, duration: 0:00:00.372963
episode: 121/200, loss: -0.0547, duration: 0:00:00.402103
episode: 131/200, loss: 0.0095, duration: 0:00:00.38

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:134: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


#############################################################################
##	[23/35]	model=RL_CNN,	SEED=0,	trade=60,	risk=1	##
#############################################################################
episode: 1/200, loss: 0.0679, duration: 0:00:00.394695
episode: 11/200, loss: -0.0052, duration: 0:00:00.388005
episode: 21/200, loss: 0.1203, duration: 0:00:00.362476
episode: 31/200, loss: -0.0041, duration: 0:00:00.361811
episode: 41/200, loss: 0.0253, duration: 0:00:00.363248
episode: 51/200, loss: -0.0384, duration: 0:00:00.390033
episode: 61/200, loss: 0.0015, duration: 0:00:00.365093
episode: 71/200, loss: 0.0176, duration: 0:00:00.365803
episode: 81/200, loss: 0.0987, duration: 0:00:00.360805
episode: 91/200, loss: -0.0036, duration: 0:00:00.364877
episode: 101/200, loss: -0.0142, duration: 0:00:00.355527
episode: 111/200, loss: -0.0433, duration: 0:00:00.378641
episode: 121/200, loss: -0.0477, duration: 0:00:00.415667
episode: 131/200, loss: 0.0099, duration: 0:00:00.3704

...

<Figure size 1008x360 with 0 Axes>

<Figure size 1008x360 with 0 Axes>

<Figure size 1008x360 with 0 Axes>

<Figure size 1008x360 with 0 Axes>

<Figure size 1008x360 with 0 Axes>

In [6]:
# double check the end of training
#env.until

In [7]:
print('training done.')

training done.


In [8]:
### Now repeat this for the different RL models.
### remember to update the gamma-pairs, model base name, and model specs/setup in beginning of notebook!